In [2]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import json
import pandas as pd

if not os.path.exists(os.path.join(os.getcwd(), 'config.json')):
  # preventing to go always to the parent folder everytime we run the jupyter notebook without restarting
  os.chdir("..")
if not os.getcwd() in sys.path:
  sys.path.append(os.getcwd())

from datavoids.find_datavoids import *
from analysis.analysis_commons import *

config = None
with open('config.json') as config_file:
  config = json.load(config_file)

config = clone_config_with_target(config, 'mit', 'dis', -1, -1)
config["database"]["database"] = "nellie_hist_crawl"

In [3]:
graph = Graph(config)
graph.connect()

graph.cursor.execute("select distinct grp from nodes where grp is not null;")
groups = graph.cursor.fetchall()
groups


[('mit',), ('dis',)]

In [4]:
graph.cursor.execute("update nodes set grp = %s where grp = %s", ('mit', groups[0]))
graph.cursor.execute("update nodes set grp = %s where grp = %s", ('dis', groups[1]))
graph.conn.commit()

In [5]:
res = find_datavoids(config, 
        min_freq_a=0.3,
        min_freq_b=0.3, 
        max_freq_ungrp=0.05,
        ratio_k=1, # penalty of freq_A(w) be different than freq_B(w).
        ratio_t=0 # influence of the ungrouped frequency
      )

df = pd.DataFrame(res, columns=['Gram', 'freq_A', 'freq_B', 'freq_U', 'ratio'])
# enable showing more results on jupyter notebook
pd.set_option('display.max_rows', 100)
df.head(100)

Updating ratio
Time: 0.00025421863344187535 minutes
done


,Gram,freq_A,freq_B,freq_U,ratio
0,funded,0.310345,0.307692,0.009009,3.600000
1,company,0.379310,0.384615,0.027027,2.750000
2,material,0.413793,0.384615,0.045045,0.545455
3,nunes,0.344828,0.384615,0.036036,0.333333
4,concerns,0.344828,0.307692,0.036036,0.285714
5,jeff,0.344828,0.307692,0.036036,0.285714
6,devin,0.310345,0.384615,0.036036,0.160714
7,author,0.310345,0.384615,0.027027,0.160714
8,congressional,0.344828,0.461538,0.045045,0.136364
9,secret,0.310345,0.461538,0.045045,0.094737


In [13]:
print(list(df['Gram']))

num_datavoids = len(df['Gram'])
freq_keywords_in_A = sum(df['freq_A']) / num_datavoids
print(f"freqA {freq_keywords_in_A:.2f}")

freq_keywords_in_B = sum(df['freq_B']) / num_datavoids
print(f"freqB {freq_keywords_in_B:.2f}")

freq_keywords_in_U = sum(df['freq_U']) / num_datavoids
print(f"freqU {freq_keywords_in_U:.2f}")

['funded', 'company', 'material', 'nunes', 'concerns', 'jeff', 'devin', 'author', 'congressional', 'secret', 'collusion']
freqA 0.34
freqB 0.41
freqU 0.04


In [6]:
graph.cursor.execute("update nodes set grp = %s where grp = %s", (groups[0], 'mit'))
graph.cursor.execute("update nodes set grp = %s where grp = %s", (groups[1], 'dis'))
graph.conn.commit()
